# Imports

In [2]:
import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import max_norm

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

# Reading Data

In [14]:
df[df['Playerno']!=1].head(7)

,Team1,Team2,Result,Match,Playerno
12,_Ali Asadinia,_Ashkan,0.55,3,1.2
13,_Alireza Najafi,_Mehdi Saadati,0.55,3,1.2
14,_Milad,_Payam Zh. Monfared,0.55,3,1.2
15,_Mohammad_Amin Ghiasi,_Sina Eslami,0.55,3,1.2
16,_Yousef Nikzad,_Amin,0.55,3,1.2
17,_Behnam Hajimosaei,NaN,0.55,3,1.2
23,_Ali Asadinia,_Afshin,0.60,5,1.2


In [5]:
df=pd.read_csv("soccer_goals_normalized.csv")
df

,Team1,Team2,Result,Match
0,_Sina Nozarian,_Behnam Hajimosaei,0.571429,1
1,_Payam Zh. Monfared,_Ali Asadinia,0.571429,1
2,_Mehdi Saadati,_Alireza Ranjbar,0.571429,1
3,_Yousef Nikzad,_Milad,0.571429,1
4,_Alireza Najafi,_Afshin,0.571429,1
...,...,...,...,...
306,_Alireza Najafi,_Jamshid,1.000000,53
307,_Payam Zh. Monfared,_Yousef Nikzad,1.000000,53
308,_Ali Asadinia,_Ali Amiri,1.000000,53
309,_Alireza Nazeri,_Ali Kalantari,1.000000,53


### Notes about the data:

The Results are derived from the following formula:

goal_ratio = ((Team_1 goals)/(Team_2 goals))/2


If the goal ratio is bigger than 2, We assign 1. if it's lesser than half, we assign 0

In other words, 0 means heavy loss for team_1 and 1 means heavy win for team_1, while 0.5 means a draw

# Data Exploration and Feature Engineering

In [6]:
d_count=df.groupby('Match').count()
d_ratio=(d_count['Team1']/d_count['Team2']).values
df['Playerno']=0
for i in range(len(d_count)):
    df.loc[df['Match']==d_count.index[i],'Playerno']=d_ratio[i]
df_r=df.copy()
df_r['Team1']=df['Team2']
df_r['Team2']=df['Team1']
df_r['Result']=1-df['Result']
df_r['Playerno']=1/df['Playerno']
df_r['Match']=df['Match']+df['Match'].max()
df=pd.concat([df,df_r])
df1=pd.concat([df,pd.get_dummies(df['Team1'])],axis=1)
df1=df1.reset_index(drop=True)
for i in df1.columns[4:]:
    df1.loc[df1['Team2']==i,i]= 0.5
df1=df1.groupby('Match',as_index=False).max()
for i in df1.columns[2:]:
    df1.loc[df1[i]==0.5,i]=-1
df1

C:\Users\User\AppData\Local\Temp\ipykernel_1844\1146439870.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['Match']==d_count.index[i],'Playerno']=d_ratio[i]
C:\Users\User\AppData\Local\Temp\ipykernel_1844\1146439870.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df1.loc[df1['Team2']==i,i]= 0.5


TypeError: agg function failed [how->max,dtype->object]

# Mean win score

In [5]:
topz=[]
dic1={}
for i in df1.columns[3:]:
    dic1[i]=df1[df1[i]==1]['Result'].mean()-0.5
Rating=dict(sorted(dic1.items(), key=lambda item: item[1]))
Rating

{'_Saleh Mohammad Rezaei': -0.25,
 '_Ali Kalantari': -0.17425925925000002,
 '_Mohammad_Amin Ghiasi': -0.06848861282608693,
 '_Behnam Hajimosaei': -0.04405643738888887,
 '_Shayan': -0.041000000000000036,
 '_Ali Amiri': -0.03623809523333332,
 '_Hadi': -0.031917293236842115,
 '_Yousef Nikzad': -0.025833333333333375,
 '_Afshin': -0.02544334975862067,
 '_Alireza Najafi': -0.025354090346153824,
 '_Ali Izadi': -0.02262626262121209,
 '_Alireza Ranjbar': -0.017817460321428502,
 '_Ashkan': -0.01749999999999996,
 '_Pedram Ensandoost': -0.008750000000000036,
 '_Alireza Nazeri': -0.005250626578947426,
 '_Ali Asadinia': 0.0009872241585365815,
 '_Amin': 0.010714285714285676,
 '_Erfan Moeini': 0.01666666666666672,
 '_Payam Zh. Monfared': 0.028515406161764734,
 '_Sina Nozarian': 0.031697994986842115,
 '_Milad': 0.037056277045454555,
 '_Mehdi Saadati': 0.07277777777777772,
 '_Jamshid': 0.07476190476666666,
 '_Sina Eslami': 0.09071428571212126,
 '_Saeid Hobbi': 0.10733333329999994,
 '_Mohsen Mirzaei': 0.

# Removing Outliers

Let's see which player has the highest played game

In [6]:
dic2={}
for i in df1.columns[3:]:
    dic2[i]=int(df1[df1[i]==1][i].sum())
dic2=dict(sorted(dic2.items(), key=lambda item: item[1]))
dic2

{'_Maghsad': 3,
 '_Saeid Hobbi': 5,
 '_Saleh Mohammad Rezaei': 5,
 '_Erfan Moeini': 6,
 '_Shayan': 10,
 '_Ali Amiri': 15,
 '_Jamshid': 15,
 '_Ali Kalantari': 18,
 '_Alireza Nazeri': 19,
 '_Hadi': 19,
 '_Ashkan': 20,
 '_Pedram Ensandoost': 20,
 '_Amin': 21,
 '_Milad': 22,
 '_Mohammad_Amin Ghiasi': 23,
 '_Yousef Nikzad': 24,
 '_Mohsen Mirzaei': 26,
 '_Behnam Hajimosaei': 27,
 '_Mehdi Saadati': 27,
 '_Afshin': 29,
 '_Ali Izadi': 33,
 '_Sina Eslami': 33,
 '_Payam Zh. Monfared': 34,
 '_Sina Nozarian': 38,
 '_Alireza Najafi': 39,
 '_Ali Asadinia': 41,
 '_Alireza Ranjbar': 42}

After talking to Sina, We decided to remove the ones with less than 8 games

In [7]:
removables=[]
for i,j in dic2.items():
    if j<8:
        removables.append(i)
df1=df1.drop(columns=removables)
df1

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1,0.571429,1.000000,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,...,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,2,0.500000,1.000000,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2,3,0.550000,1.200000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,1.0
3,4,0.550000,1.000000,-1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,0.600000,1.200000,-1.0,0.0,1.0,-1.0,-1.0,1.0,-1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,102,0.500000,0.833333,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0
102,103,0.475000,1.000000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0
103,104,0.000000,1.000000,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0
104,105,0.000000,1.000000,1.0,1.0,-1.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,1.0,1.0,-1.0,1.0,1.0,-1.0,0.0,0.0


### Mean Score now:

In [8]:
topz=[]
dic1={}
for i in df1.columns[3:]:
    dic1[i]=df1[df1[i]==1]['Result'].mean()-0.5
Rating=dict(sorted(dic1.items(), key=lambda item: item[1]))
Rating

{'_Ali Kalantari': -0.17425925925000002,
 '_Mohammad_Amin Ghiasi': -0.06848861282608693,
 '_Behnam Hajimosaei': -0.04405643738888887,
 '_Shayan': -0.041000000000000036,
 '_Ali Amiri': -0.03623809523333332,
 '_Hadi': -0.031917293236842115,
 '_Yousef Nikzad': -0.025833333333333375,
 '_Afshin': -0.02544334975862067,
 '_Alireza Najafi': -0.025354090346153824,
 '_Ali Izadi': -0.02262626262121209,
 '_Alireza Ranjbar': -0.017817460321428502,
 '_Ashkan': -0.01749999999999996,
 '_Pedram Ensandoost': -0.008750000000000036,
 '_Alireza Nazeri': -0.005250626578947426,
 '_Ali Asadinia': 0.0009872241585365815,
 '_Amin': 0.010714285714285676,
 '_Payam Zh. Monfared': 0.028515406161764734,
 '_Sina Nozarian': 0.031697994986842115,
 '_Milad': 0.037056277045454555,
 '_Mehdi Saadati': 0.07277777777777772,
 '_Jamshid': 0.07476190476666666,
 '_Sina Eslami': 0.09071428571212126,
 '_Mohsen Mirzaei': 0.14717032967307686}

### Looks Good

In [9]:
list_of_players=df1.columns[3:]

In [10]:
list_of_players

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei', '_Hadi',
       '_Jamshid', '_Mehdi Saadati', '_Milad', '_Mohammad_Amin Ghiasi',
       '_Mohsen Mirzaei', '_Payam Zh. Monfared', '_Pedram Ensandoost',
       '_Shayan', '_Sina Eslami', '_Sina Nozarian', '_Yousef Nikzad'],
      dtype='object')

# Pairs Analysis

### Get every pair:

In [11]:
pair_of_players=[]
for c,i in enumerate(list_of_players):
    for x in range(len(list_of_players)):
        if x>c:
            pair_of_players.append([i,list_of_players[x]])
    

In [12]:
pair_of_players[-1]

['_Sina Nozarian', '_Yousef Nikzad']

### Extract stats for every pair:

In [13]:
def get_pair_winrate(a,b,df1):
    mean=df1[(df1[a]==1)&(df1[b]==1)]['Result'].mean()
    count=df1[(df1[a]==1)&(df1[b]==1)]['Result'].count()
    median=df1[(df1[a]==1)&(df1[b]==1)]['Result'].median()
    return mean,median,count

In [14]:
dr=pd.DataFrame()
for i in pair_of_players:
    mean,median,count=get_pair_winrate(i[0],i[1],df1)
    dr=dr.append({'player1':i[0],'player2':i[1],'mean':mean,'median':median,'count':count},ignore_index=True)
    dr=dr.dropna(how='any')
dr

,player1,player2,mean,median,count
0,_Afshin,_Ali Amiri,0.330000,0.490000,3.0
1,_Afshin,_Ali Asadinia,0.478143,0.450000,15.0
2,_Afshin,_Ali Izadi,0.490625,0.500000,16.0
3,_Afshin,_Ali Kalantari,0.321000,0.400000,5.0
4,_Afshin,_Alireza Najafi,0.336250,0.172500,4.0
...,...,...,...,...,...
231,_Shayan,_Sina Nozarian,0.558333,0.525000,3.0
232,_Shayan,_Yousef Nikzad,0.432500,0.432500,2.0
233,_Sina Eslami,_Sina Nozarian,0.725000,1.000000,3.0
234,_Sina Eslami,_Yousef Nikzad,0.563393,0.476786,4.0


Before we start, let's remove the ones with low counts, or make a new parameter that scores based on number of games

## Setting Score

In [15]:
dr['pair_score']=((dr['median']-0.5)*10+(dr['mean']-0.5)*10)*(np.log(dr['count']+15)-2.83)**1

In [16]:
dr['pair_score'].describe()

count    236.000000
mean       0.013361
std        0.595357
min       -2.139185
25%       -0.235614
50%        0.000000
75%        0.250478
max        2.877068
Name: pair_score, dtype: float64

# Most frequent pairs:

In [17]:
dr.sort_values('count',ascending=False).head(10)

,player1,player2,mean,median,count,pair_score
58,_Ali Asadinia,_Sina Nozarian,0.526731,0.500000,26.0,0.236186
45,_Ali Asadinia,_Alireza Ranjbar,0.491071,0.500000,22.0,-0.069725
136,_Alireza Ranjbar,_Sina Nozarian,0.539655,0.500000,20.0,0.287635
218,_Mohsen Mirzaei,_Sina Eslami,0.663158,0.750000,19.0,2.877068
41,_Ali Asadinia,_Ali Izadi,0.358958,0.316667,16.0,-1.959183
132,_Alireza Ranjbar,_Payam Zh. Monfared,0.438943,0.500000,16.0,-0.368774
2,_Afshin,_Ali Izadi,0.490625,0.500000,16.0,-0.056624
124,_Alireza Ranjbar,_Ashkan,0.509375,0.500000,16.0,0.056624
59,_Ali Asadinia,_Yousef Nikzad,0.437905,0.450000,15.0,-0.640285
76,_Ali Izadi,_Sina Nozarian,0.437667,0.450000,15.0,-0.641645


# Highest pair scores

In [18]:

dr=dr.sort_values('pair_score',ascending=False)
dr=dr.reset_index(drop=True)
dr.head(15)

,player1,player2,mean,median,count,pair_score
0,_Mohsen Mirzaei,_Sina Eslami,0.663158,0.750000,19.0,2.877068
1,_Pedram Ensandoost,_Sina Eslami,0.697273,0.750000,11.0,1.914759
2,_Alireza Najafi,_Mehdi Saadati,0.616369,0.750000,12.0,1.706682
3,_Alireza Najafi,_Mohsen Mirzaei,0.642857,0.750000,11.0,1.681808
4,_Mehdi Saadati,_Pedram Ensandoost,0.722222,0.750000,9.0,1.643588
5,_Alireza Najafi,_Pedram Ensandoost,0.628636,0.750000,11.0,1.620929
6,_Mohsen Mirzaei,_Pedram Ensandoost,0.590909,0.750000,11.0,1.459420
7,_Mehdi Saadati,_Mohsen Mirzaei,0.666667,0.750000,9.0,1.450224
8,_Milad,_Sina Nozarian,0.628788,0.666667,11.0,1.264831
9,_Ali Amiri,_Mohsen Mirzaei,0.607347,0.830000,7.0,1.141661


In [19]:
dr[(dr['player1']=='_Milad')|(dr['player2']=='_Milad')]['count'].sum()

113.0

# Least Pair reliant players

In [20]:
np.log(43)

3.7612001156935624

In [21]:
dp=(dr.groupby('player1')['pair_score'].agg(['max','min'])+dr.groupby('player2')['pair_score'].agg(['max','min']))/2
#Adding count:
dp['count']=0
for i in dp.index.values:
    count=dr[(dr['player1']==i)|(dr['player2']==i)]['count'].sum()
    dp.loc[dp.index.values==i,'count']=count

dp.loc['_Afshin','min']=dr.groupby('player1')['pair_score'].agg(['max','min'])['min'][0]
dp.loc['_Afshin','max']=dr.groupby('player1')['pair_score'].agg(['max','min'])['max'][0]

dp.loc['_Yousef Nikzad','min']=dr.groupby('player2')['pair_score'].agg(['max','min'])['min'][-1]
dp.loc['_Yousef Nikzad','max']=dr.groupby('player2')['pair_score'].agg(['max','min'])['max'][-1]
dp['reliant_factor']=((dp['max']-5*dp['min'])-(np.log(dp['count']))*0.4)/10
dp=dp.sort_values('reliant_factor')
dp

,max,min,count,reliant_factor
_Milad,1.146762,-0.150794,113,0.000977
_Shayan,0.226394,-0.289225,53,0.008440
_Mehdi Saadati,1.675135,-0.069339,119,0.011018
_Sina Nozarian,0.587972,-0.365265,176,0.034611
_Sina Eslami,1.657382,-0.245733,154,0.087127
_Hadi,0.386972,-0.481783,103,0.094199
_Ashkan,0.383738,-0.503838,82,0.114024
_Amin,0.432885,-0.588670,93,0.156319
_Jamshid,0.745427,-0.510517,70,0.159861
_Ali Amiri,0.516496,-0.626529,78,0.190646


# Best pair for each player:

In [22]:
dr1=dr[dr['count']>2]
colz=['Player','Best_Match','Worst_Match']
colz.extend(dr.columns[2:].tolist())
dp2=pd.DataFrame(columns=colz)
dp2
for i in list_of_players:
    #Max:
    d_temp=dr1[(dr1['player1']==i)|(dr1['player2']==i)].values[0]
    first_name=d_temp[0]
    second_name=d_temp[1]
    if first_name!=i:
              Best_name=first_name
    else:
              Best_name=second_name
    #Min:
    d_temp_2=dr1[(dr1['player1']==i)|(dr1['player2']==i)].values[-1]
    first_name_worst=d_temp_2[0]
    second_name_worst=d_temp_2[1]
    if first_name_worst!=i:
              Worst_name=first_name_worst
    else:
              Worst_name=second_name_worst
    
    
    dp2=dp2.append({'Player':i,'Best_Match':Best_name,'Worst_Match':np.nan,'mean':d_temp[2],'median':d_temp[3],'count':d_temp[4],'pair_score':d_temp[5]},ignore_index=True)
    dp2=dp2.append({'Player':i,'Best_Match':np.nan,'Worst_Match':Worst_name,'mean':d_temp_2[2],'median':d_temp_2[3],'count':d_temp_2[4],'pair_score':d_temp_2[5]},ignore_index=True)
dp2

,Player,Best_Match,Worst_Match,mean,median,count,pair_score
0,_Afshin,_Sina Nozarian,NaN,0.601429,0.585000,7.0,0.486658
1,_Afshin,NaN,_Payam Zh. Monfared,0.298000,0.000000,5.0,-1.163441
2,_Ali Amiri,_Mohsen Mirzaei,NaN,0.607347,0.830000,7.0,1.141661
3,_Ali Amiri,NaN,_Pedram Ensandoost,0.000000,0.000000,4.0,-1.144390
4,_Ali Asadinia,_Payam Zh. Monfared,NaN,0.593750,0.750000,8.0,1.050136
5,_Ali Asadinia,NaN,_Ali Izadi,0.358958,0.316667,16.0,-1.959183
6,_Ali Izadi,_Mohsen Mirzaei,NaN,0.640500,0.625000,10.0,1.032465
7,_Ali Izadi,NaN,_Ali Asadinia,0.358958,0.316667,16.0,-1.959183
8,_Ali Kalantari,_Sina Nozarian,NaN,0.528333,0.525000,3.0,0.032198
9,_Ali Kalantari,NaN,_Alireza Najafi,0.250303,0.250000,11.0,-2.139185


# Enabler

In [23]:
dp2['Best_Match'].value_counts()

_Mohsen Mirzaei        5
_Sina Nozarian         3
_Payam Zh. Monfared    3
_Sina Eslami           3
_Milad                 3
_Mehdi Saadati         2
_Jamshid               1
_Amin                  1
_Alireza Najafi        1
_Ali Asadinia          1
Name: Best_Match, dtype: int64

# Disabler

In [24]:
dp2['Worst_Match'].value_counts()

_Pedram Ensandoost       4
_Alireza Najafi          3
_Mohammad_Amin Ghiasi    3
_Payam Zh. Monfared      2
_Ali Izadi               2
_Ali Kalantari           2
_Behnam Hajimosaei       2
_Ali Asadinia            1
_Amin                    1
_Sina Nozarian           1
_Ashkan                  1
_Afshin                  1
Name: Worst_Match, dtype: int64

# Worst Opponents

In [25]:
df1

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1,0.571429,1.000000,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,...,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,2,0.500000,1.000000,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2,3,0.550000,1.200000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,1.0
3,4,0.550000,1.000000,-1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,0.600000,1.200000,-1.0,0.0,1.0,-1.0,-1.0,1.0,-1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,102,0.500000,0.833333,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0
102,103,0.475000,1.000000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0
103,104,0.000000,1.000000,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0
104,105,0.000000,1.000000,1.0,1.0,-1.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,1.0,1.0,-1.0,1.0,1.0,-1.0,0.0,0.0


# Splitz:

In [26]:
x_train, x_test, y_train, y_test = train_test_split(df1.drop(columns=['Result','Match']).values, df1['Result'].values, test_size=0.3, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

x_train,y_train=df1.drop(columns=['Result','Match']).values, df1['Result'].values
x_test,y_test=x_train,y_train

(74, 24)
(74,)
(32, 24)
(32,)


#### For All data:

In [27]:
# x_train=df1.drop(columns=['Result','Match']).values
# y_train=df1['Result'].values
# print(x_train.shape)
# print(y_train.shape)

# Modeling

# Deep

# Create model

In [28]:
class SingleAttention(Layer):
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, 
                           input_shape=input_shape, 
#                            kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2),
#                            activation='relu',
                           kernel_initializer='glorot_uniform', 
                           bias_initializer='glorot_uniform')

        self.key = Dense(self.d_k, 
                         input_shape=input_shape, 
#                          kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2),
#                          activation='relu',
                         kernel_initializer='glorot_uniform', 
                         bias_initializer='glorot_uniform')

        self.value = Dense(self.d_v, 
                           input_shape=input_shape, 
#                            kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2),
#                            activation='relu',
                           kernel_initializer='glorot_uniform', 
                           bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)

        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out    
    
    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'d_k': self.d_k,
                       'd_v': self.d_v,
                      })
        return config     

## Normal Model

In [29]:
def create_model_normal(lr1):
#     attn_layer1 = SingleAttention(8,8)
#     normalizer=LayerNormalization(input_shape=(8,8), epsilon=1e-6)
#     attn_layer2 = SingleAttention(26,26)
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
    x = Dropout(0.3)(x)
    x = Dense(14, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.3)(x)
    x = Dense(14, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.3)(x)
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.4)(x)
#     x = Dense(11, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=1), bias_constraint=max_norm(max_value=1))(x)
#     x = Dropout(0.4)(x)
    out = Dense(1,activation='sigmoid')(x)

    
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)

    return model


model1 = create_model_normal(0.0001)
# model2.load_weights('soccer2_Attention.h5')
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense (Dense)               (None, 14)                350       
                                                                 
 dropout_1 (Dropout)         (None, 14)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                210       
                                                                 
 dropout_2 (Dropout)         (None, 14)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 120   

/anaconda/envs/azureml_py38_tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


## Attention Model

In [30]:
def create_model(lr1):
    attn_layer1 = SingleAttention(14,14)
    attn_layer2 = SingleAttention(14,14)
    normalizer=LayerNormalization(input_shape=(8,8), epsilon=1e-6)
    
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
    x = Dense(14, activation='tanh',kernel_initializer='glorot_uniform')(x)
#     x = Dropout(0.2)(x)
#     x = Dense(14, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2))(x)
    
    x1 =x
    x = Dropout(0.3)(x)
    x = attn_layer1((x,x,x))
    x= normalizer(x+x1)  
    x=x+x1
    x1 =x
    x = Dropout(0.3)(x)
    x = attn_layer2((x,x,x))
    x= normalizer(x+x1)
    x=x+x1
    x = Dropout(0.3)(x)
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2))(x)
    x = Dropout(0.4)(x)
    out = Dense(1,activation='sigmoid')(x)

    
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)

    return model


model2 = create_model(0.0001)
model2.load_weights('soccer2_Attention_dimensionR_2_edited.h5')
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 14)           350         ['input_2[0][0]']                
                                                                                                  
 dropout_4 (Dropout)            (None, 14)           0           ['dense_4[0][0]']                
                                                                                                  
 single_attention (SingleAttent  (None, 14)          630         ['dropout_4[0][0]',              
 ion)                                                             'dropout_4[0][0]',        

In [31]:
# lr1=0.001
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
# model.compile(loss= keras.losses.BinaryCrossentropy(), 
#               optimizer=optimizer)
# h=model.fit(x_train,y_train,epochs=5000,batch_size=2,shuffle=True,)

# Encode

In [32]:
def create_model_encode(lr1):
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
#     x = Dropout(0.2)(x)
    x = Dense(14,activation='tanh',kernel_initializer='glorot_uniform',name='Encoded')(x)#,kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3)
#     x = Dropout(0.2)(x)
    out = Dense(x_train.shape[1],activation='tanh')(x)
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.MeanAbsoluteError(), #.MeanAbsoluteError(),
                  optimizer=optimizer)

    return model


model3 = create_model_encode(0.001)
model3.summary()
# model3.fit(x_train,x_train,epochs=1000,batch_size=16,shuffle=True)
# model3.save('Encode.h5',include_optimizer=False)
# model3.load_weights('Encode.h5')
# # Weights(Untrainable)
def w_load(layer1,layer2,train_layer=True):
    if len(layer1.get_weights())>0:
        layer1.set_weights(layer2.get_weights())
        if train_layer==False:
            layer1.trainable=False



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 24)]              0         
                                                                 
 Encoded (Dense)             (None, 14)                350       
                                                                 
 dense_7 (Dense)             (None, 24)                360       
                                                                 
Total params: 710
Trainable params: 710
Non-trainable params: 0
_________________________________________________________________


In [33]:
print(x_train[1])
model3.predict(np.reshape(x_train[1],(1,x_train.shape[1])))

[ 1. -1.  0.  1.  1.  0.  0.  0.  1.  0. -1.  0. -1.  0. -1.  0.  0. -1.
  1.  0.  0. -1.  1.  0.]


array([[ 0.49781302, -0.32523832, -0.5724393 , -0.43158716, -0.44893077,
        -0.04971398, -0.06803991, -0.36621583, -0.18506755, -0.18498684,
        -0.66420215, -0.5306681 ,  0.12458979, -0.55988926, -0.12990925,
        -0.5179851 , -0.07115723, -0.29313102,  0.559983  , -0.53283644,
         0.13776319,  0.17244887, -0.4391019 , -0.5503112 ]],
      dtype=float32)

## Train

#### Encode

In [42]:
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=0.0001)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model1.compile(loss= keras.losses.MeanAbsoluteError(), 
              optimizer=optimizer)
model3.fit(x_train,x_train,epochs=1500,batch_size=16,shuffle=True)
model3.save('Encode.h5',include_optimizer=False)

Epoch 1/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0388
Epoch 2/1500
7/7 [==============================] - 0s 982us/step - loss: 0.0391
Epoch 3/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0390
Epoch 4/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0390
Epoch 5/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0389
Epoch 6/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0390
Epoch 7/1500
7/7 [==============================] - 0s 989us/step - loss: 0.0391
Epoch 8/1500
7/7 [==============================] - 0s 986us/step - loss: 0.0391
Epoch 9/1500
7/7 [==============================] - 0s 994us/step - loss: 0.0392
Epoch 10/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0390
Epoch 11/1500
7/7 [==============================] - 0s 1ms/step - loss: 0.0389
Epoch 12/1500
7/7 [==============================] - 0s 982us/step - loss: 0.0391
Epoch 13/1500
7/7 [====================

#### Model 1

In [46]:
w_load(model1.layers[2],model3.layers[1],train_layer=False)   
lr1=0.001
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model1.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer)
model1.summary()
# model1.load_weights('soccer2_normal.h5')
with tf.device('/cpu:0'):
    
    h=model1.fit(x_train,y_train,epochs=500,batch_size=16,shuffle=True)
#     model1.save('soccer2_normal_2.h5',include_optimizer=False)
    


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense (Dense)               (None, 14)                350       
                                                                 
 dropout_1 (Dropout)         (None, 14)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                210       
                                                                 
 dropout_2 (Dropout)         (None, 14)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 120   

#### Model 2

In [50]:
w_load(model2.layers[1],model3.layers[1],train_layer=False)    
lr1=0.0001
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model2.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer)
# model2.load_weights('soccer2_Attention_dimensionR_2.h5')
with tf.device('/cpu:0'):
    
    h=model2.fit(x_train,y_train,epochs=200,batch_size=16,shuffle=True)
#     model2.save('soccer2_Attention_dimensionR_2.h5',include_optimizer=False)

Epoch 1/200
7/7 [==============================] - 2s 5ms/step - loss: 0.5998
Epoch 2/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6045
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6224
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6149
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6088
Epoch 6/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6013
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6160
Epoch 8/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6019
Epoch 9/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6041
Epoch 10/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6156
Epoch 11/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6197
Epoch 12/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6286
Epoch 13/200
7/7 [==============================] - 0s 4ms/st

In [51]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 14)           350         ['input_2[0][0]']                
                                                                                                  
 dropout_4 (Dropout)            (None, 14)           0           ['dense_4[0][0]']                
                                                                                                  
 single_attention (SingleAttent  (None, 14)          630         ['dropout_4[0][0]',              
 ion)                                                             'dropout_4[0][0]',        

In [64]:
with tf.device('/cpu:0'):
    model2.fit(x_train,y_train,epochs=10,batch_size=16,shuffle=True)

Epoch 1/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6560
Epoch 2/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6427
Epoch 3/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6350
Epoch 4/10
7/7 [==============================] - 0s 6ms/step - loss: 0.6455
Epoch 5/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6234
Epoch 6/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6429
Epoch 7/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6166
Epoch 8/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6310
Epoch 9/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6277
Epoch 10/10
7/7 [==============================] - 0s 4ms/step - loss: 0.6380


In [60]:
print('Model 2')
model2.evaluate(x_test,y_test)
model2.evaluate(x_train,y_train)
print('Model 1')
model1.evaluate(x_test,y_test)
model1.evaluate(x_train,y_train)

Model 2
4/4 [==============================] - 0s 2ms/step - loss: 0.5601
Model 1
4/4 [==============================] - 0s 1ms/step - loss: 0.6401


0.6400628685951233

## Retrain:

In [31]:
# with tf.device("/cpu:1"):
#     model2=keras.models.load_model('soccer2_Attention_dimensionR.h5',compile=False)

#### Train last xx matches

In [58]:
def train_latest_data(x_train,y_train,model,latest=[1],lr1=0.0001,epoch=1000):
#Picking 2 records of latest match:
    indexes_1=[]
    indexes_2=[]
    for j in latest:
        indexes_1.append(len(x_train)//2-j)
        indexes_2.append(len(x_train)-j)
    all_indexes=indexes_1
    all_indexes.extend(indexes_2)
    
    x_train_n=x_train[all_indexes]
    y_train_n=y_train[all_indexes]

    optimizer=keras.optimizers.SGD(lr=lr1,momentum=0.2)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)
    h=model.fit(x_train_n,y_train_n,epochs=epoch,batch_size=16,shuffle=True)
    
# for i in range(1,6):
train_latest_data(x_train,y_train,model2,latest=[i for i in range(1,30)],epoch=2000)

Epoch 1/2000
4/4 [==============================] - 1s 5ms/step - loss: 0.5645
Epoch 2/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5623
Epoch 3/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5568
Epoch 4/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5109
Epoch 5/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5479
Epoch 6/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.6089
Epoch 7/2000
4/4 [==============================] - 0s 3ms/step - loss: 0.5378
Epoch 8/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5585
Epoch 9/2000
4/4 [==============================] - 0s 3ms/step - loss: 0.5240
Epoch 10/2000
4/4 [==============================] - 0s 3ms/step - loss: 0.5324
Epoch 11/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5607
Epoch 12/2000
4/4 [==============================] - 0s 4ms/step - loss: 0.5136
Epoch 13/2000
4/4 [==============================

# Saves

In [59]:
model2.save('soccer2_Attention_dimensionR_2_edited.h5',include_optimizer=False)

In [397]:
model.save('soccer2.h5',include_optimizer=False)

# Monitoring results

#### Model 1

In [61]:
p=model1.predict(x_test)
dr=pd.DataFrame()
dr['Predicted']=np.reshape(p,p.shape[0])
dr['Real']=y_test
diff=(abs(dr['Real']-dr['Predicted'])*100)**2
print(sum(diff[-25:]))
dr

20526.910304768062


,Predicted,Real
0,0.525293,0.571429
1,0.409580,0.500000
2,0.423554,0.550000
3,0.580594,0.550000
4,0.463666,0.600000
...,...,...
101,0.495825,0.500000
102,0.399214,0.475000
103,0.356630,0.000000
104,0.413691,0.000000


In [62]:
p=model2.predict(x_test)
dr=pd.DataFrame()
dr['Predicted']=np.reshape(p,p.shape[0])
dr['Real']=y_test
diff=(abs(dr['Real']-dr['Predicted'])*100)**2
print(sum(diff[-25:]))
dr

5284.747366728392


,Predicted,Real
0,0.605080,0.571429
1,0.436219,0.500000
2,0.568378,0.550000
3,0.575004,0.550000
4,0.506726,0.600000
...,...,...
101,0.489135,0.500000
102,0.416505,0.475000
103,0.142739,0.000000
104,0.150448,0.000000


# classic models

In [41]:
# df1['Result']=df1['Result']-0.5

In [63]:
regr = RandomForestRegressor(max_depth=5, random_state=0,n_estimators=15)
# regr.fit(df1.drop(columns=['Result','Match']).values[:-2], df1['Result'][:-2])
regr.fit(x_train, y_train)
# print(regr.predict([[0, 0, 0, 0]]))

RandomForestRegressor(max_depth=5, n_estimators=15, random_state=0)

In [64]:
reg = LinearRegression().fit(df1.drop(columns=['Result','Match']).values, df1['Result'])

In [65]:
p2=regr.predict(x_test)
dr2=pd.DataFrame()
dr2['Predicted']=p2
dr2['Real']=y_test
diff=((dr2['Real']-dr2['Predicted'])*100)**2
print(sum(diff[-25:]))
dr2

7960.918155682317


,Predicted,Real
0,0.526411,0.571429
1,0.451555,0.500000
2,0.512107,0.550000
3,0.614654,0.550000
4,0.660498,0.600000
...,...,...
101,0.406698,0.500000
102,0.458581,0.475000
103,0.097342,0.000000
104,0.161636,0.000000


# Importance

In [66]:
rate=dict(zip(df1.columns[2:],regr.feature_importances_))
dict(sorted(rate.items(), key=lambda item: item[1]))

# rate=dict(zip(df1.columns[2:],reg.coef_))
# dict(sorted(rate.items(), key=lambda item: item[1]))

{'_Amin': 0.007937961145463236,
 '_Alireza Najafi': 0.00844339605794388,
 '_Behnam Hajimosaei': 0.008712031517937875,
 '_Hadi': 0.009939441694054117,
 '_Payam Zh. Monfared': 0.01105487635534459,
 '_Yousef Nikzad': 0.01935662929698209,
 '_Mohammad_Amin Ghiasi': 0.019846456736381874,
 '_Shayan': 0.021464148242694808,
 '_Mehdi Saadati': 0.023981617282916253,
 'Playerno': 0.024651089009198806,
 '_Ali Asadinia': 0.02653041983029,
 '_Jamshid': 0.026851405883508435,
 '_Alireza Nazeri': 0.02686746872688428,
 '_Ashkan': 0.0287158873328342,
 '_Sina Nozarian': 0.028936722697910772,
 '_Ali Izadi': 0.029571944804597884,
 '_Alireza Ranjbar': 0.03671046074393428,
 '_Ali Amiri': 0.03939701697773414,
 '_Afshin': 0.04730198376044016,
 '_Milad': 0.05236837285817735,
 '_Sina Eslami': 0.058230311938058395,
 '_Pedram Ensandoost': 0.05946674038158248,
 '_Ali Kalantari': 0.15288824224720846,
 '_Mohsen Mirzaei': 0.23077537447792162}

# Mean win

In [67]:
topz=[]
dic1={}
for i in df1.columns[3:]:
    dic1[i]=df1[df1[i]==1]['Result'].mean()-0.5

In [68]:
Rating=dict(sorted(dic1.items(), key=lambda item: item[1]))

In [69]:
Rating

{'_Ali Kalantari': -0.17425925925000002,
 '_Mohammad_Amin Ghiasi': -0.06848861282608693,
 '_Behnam Hajimosaei': -0.04405643738888887,
 '_Shayan': -0.041000000000000036,
 '_Ali Amiri': -0.03623809523333332,
 '_Hadi': -0.031917293236842115,
 '_Yousef Nikzad': -0.025833333333333375,
 '_Afshin': -0.02544334975862067,
 '_Alireza Najafi': -0.025354090346153824,
 '_Ali Izadi': -0.02262626262121209,
 '_Alireza Ranjbar': -0.017817460321428502,
 '_Ashkan': -0.01749999999999996,
 '_Pedram Ensandoost': -0.008750000000000036,
 '_Alireza Nazeri': -0.005250626578947426,
 '_Ali Asadinia': 0.0009872241585365815,
 '_Amin': 0.010714285714285676,
 '_Payam Zh. Monfared': 0.028515406161764734,
 '_Sina Nozarian': 0.031697994986842115,
 '_Milad': 0.037056277045454555,
 '_Mehdi Saadati': 0.07277777777777772,
 '_Jamshid': 0.07476190476666666,
 '_Sina Eslami': 0.09071428571212126,
 '_Mohsen Mirzaei': 0.14717032967307686}

# Pearson Correlation

In [70]:
feat=df1.corr('pearson')[['Result']]

feat['Result']=abs(feat['Result'])
feat[2:].sort_values('Result',ascending=False)

,Result
_Mohsen Mirzaei,0.349515
_Ali Kalantari,0.344343
_Sina Eslami,0.246548
_Mehdi Saadati,0.176132
_Mohammad_Amin Ghiasi,0.152982
Playerno,0.144744
_Jamshid,0.139688
_Behnam Hajimosaei,0.106623
_Sina Nozarian,0.091009
_Milad,0.080953


# Pipeline

In [71]:
names=df1.columns[3:]
names

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei', '_Hadi',
       '_Jamshid', '_Mehdi Saadati', '_Milad', '_Mohammad_Amin Ghiasi',
       '_Mohsen Mirzaei', '_Payam Zh. Monfared', '_Pedram Ensandoost',
       '_Shayan', '_Sina Eslami', '_Sina Nozarian', '_Yousef Nikzad'],
      dtype='object')

In [76]:
team1=['_Pedram Ensandoost','_Sina Eslami','_Payam Zh. Monfared','_Yousef Nikzad','_Ali Kalantari','_Jamshid']#,'_Behnam Hajimosaei'
team2=['_Sina Nozarian','_Ali Asadinia','_Alireza Ranjbar','_Hadi','_Mohammad_Amin Ghiasi','_Alireza Najafi']#
,
print(len(team1))
print(len(team2))

6
6


In [74]:
def get_x_ready(team1,team2,has__=False):
    if has__==False:
        teams1=['_'+i for i in team1]
        teams2=['_'+i for i in team2]
    else:
        teams1=team1
        teams2=team2
#     teams=teams1
#     teams.extend(teams2)
#     teams=np.array(teams)
    dt=pd.DataFrame(columns=names)
    dt.loc[0]=np.zeros(len(names))#All zero
    dt[teams1]=1
    dt[teams2]=-1
    return dt.values

In [75]:
def predict_game(model,regr,team1,team2,has__=False,print_stuff=True):
#     with tf.device('/cpu:0'):
    teamz1=team1
    teamz2=team2
    playerno=len(teamz1)/len(teamz2)
    x1=np.expand_dims(np.append(playerno,get_x_ready(teamz1,teamz2,has__)),0)
    
    #Predict,swap,predict
    rez1=model.predict(x1)[0][0]
    rez2=regr.predict(x1)[0]
    
    #Swap
    x2=np.expand_dims(np.append(1/playerno,get_x_ready(teamz2,teamz1,has__)),0)

    rez11=1-model.predict(x2)[0][0]#1-
    rez22=1-(regr.predict(x2)[0])    
    rez_final=(rez1+rez11)/2
    rez1=round((rez1+rez11)/2,2)
    rez2=round((rez2+rez22)/2,2)
    if print_stuff:
        #Deep:
        if rez1>0.55:
            print(f'Deep model prediction: Team 1 will Win with a probability of: {rez1}')
        elif rez1<0.45:
            print(f'Deep model prediction: Team 2 will Win with a probability of: {1-rez1}')
        else:
            print(f"Deep model prediction: Likely Draw, with {rez1} chance of Team 1 winning")

        #Classic:
        if rez2>0.55:
            print(f'Classic model prediction: Team 1 will Win with a probability of: {rez2}')
        elif rez2<0.45:
            print(f'Classic model prediction: Team 2 will Win with a probability of: {1-rez2}')
        else:
            print(f"Classic model prediction: Likely Draw, with {rez2} chance of Team 1 winning")    
    
    return rez_final
    

In [77]:
predict_game(model2,regr,team1,team2,has__=True)

Deep model prediction: Likely Draw, with 0.49 chance of Team 1 winning
Classic model prediction: Team 2 will Win with a probability of: 0.64


0.4865371137857437

In [89]:
names

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei', '_Hadi',
       '_Jamshid', '_Mehdi Saadati', '_Milad', '_Mohammad_Amin Ghiasi',
       '_Mohsen Mirzaei', '_Payam Zh. Monfared', '_Pedram Ensandoost',
       '_Shayan', '_Sina Eslami', '_Sina Nozarian', '_Yousef Nikzad'],
      dtype='object')

# All matchups

Pick 5 players for every possible scenario:

# Pick players

So we have 8 spots. we need all the variations

In [243]:
list_of_candidates=['_Afshin','_Ali Asadinia','_Ali Izadi','_Sina Eslami','_Payam Zh. Monfared','_Mohsen Mirzaei','_Milad', '_Alireza Najafi']
print(len(list_of_candidates))

8


In [244]:
def pick_teams(players):
    counter=0
    teams=[]
    team1=[]
    team2=[]
    playholder=players.copy()
#     df2=df.copy()
    for c1,i in enumerate(playholder):#First
        team1.append(i)
        playholder=players.copy()
        playholder.remove(i)
        for c2,j in enumerate(playholder):#Second
            team1.append(j)
            playholder=players.copy()
            playholder.remove(i)
            playholder.remove(j)
            for c3,k in enumerate(playholder):#Third
                playholder=players.copy()
                playholder.remove(k)
                playholder.remove(j)
                playholder.remove(i)
                team1.append(k)
                for c4,z in enumerate(playholder):#Fourth
                    playholder=players.copy()
                    playholder.remove(z)
                    playholder.remove(k)
                    playholder.remove(j)
                    playholder.remove(i)
                    team1.append(z)
#                     print(playholder)
                    team_1=team1.copy()
                    teams.append([team_1,playholder])
                    counter=counter+1

                    team1.remove(z)
                team1.remove(k)
            team1.remove(j)
        team1.remove(i)
    return teams
#                     print(team2)
team_list=pick_teams(list_of_candidates)
# print("ANSWER:")
# returned
            
    

Now that we have the entire list, we can predict the result and put it all in a DF

In [400]:
x_array=np.zeros((1,26))
for c,i in enumerate(team_list):
    team1=['_Sina Nozarian']
    team1.append(team_list[c][0])
    team2=['_Amin']
    team2.append(team_list[c][1])
    y=predict_game(model2,regr,team1,team2,has__=True,print_stuff=False)
    x=np.expand_dims(np.append([1,y,1],get_x_ready(team1,team2,has__=True)),0)
    x_array=np.append(x_array,x,0)

for c,i in enumerate(team_list):
    team1=['_Amin']
    team1.append(team_list[c][0])
    team2=['_Sina Nozarian']
    team2.append(team_list[c][1])
    y=predict_game(model2,regr,team1,team2,has__=True,print_stuff=False)
    x=np.expand_dims(np.append([1,y,1],get_x_ready(team1,team2,has__=True)),0)
    x_array=np.append(x_array,x,0)
    
dt=pd.DataFrame(x_array[1:],columns=df1.columns)



In [461]:
dt=pd.DataFrame(x_array[1:],columns=df1.columns)

In [462]:
dt=dt.drop_duplicates()
dt=dt.reset_index(drop=True)
# dt['Match']=dt.index.values+1
dt

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1.0,0.473934,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.389223,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2,1.0,0.659868,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
3,1.0,0.349747,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
4,1.0,0.153501,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1.0,0.846499,1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0
136,1.0,0.650253,1.0,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0
137,1.0,0.340132,1.0,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,1.0,-1.0,0.0
138,1.0,0.610777,1.0,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,-1.0,0.0,0.0,1.0,-1.0,0.0


In [468]:
x_inversed=np.where(abs(x_array[1:])==1,x_array[1:]*-1,x_array[1:])
dt2=pd.DataFrame(x_inversed,columns=df1.columns)
dt2=dt2.drop_duplicates()
# dt2=dt2.reset_index(drop=True)
dt2['Playerno']=1
dt2['Result']=1-dt2['Result']
dt2

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,-1.0,0.526066,1,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,-1.0,-1.0,0.0
1,-1.0,0.610777,1,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,-1.0,0.0,0.0,1.0,-1.0,0.0
2,-1.0,0.340132,1,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,1.0,-1.0,0.0
3,-1.0,0.650253,1,-1.0,0.0,-1.0,-1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0
4,-1.0,0.846499,1,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,-1.0,0.153501,1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
2419,-1.0,0.349747,1,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
2424,-1.0,0.659868,1,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
2454,-1.0,0.389223,1,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0


In [469]:
dt3=pd.concat([dt,dt2])
dt3=dt3.reset_index(drop=True)
dt3['Match']=1
dt3['Match']=dt3.index.values+1
dt3

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1,0.473934,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,1.0,1.0,0.0
1,2,0.389223,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2,3,0.659868,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
3,4,0.349747,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
4,5,0.153501,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,276,0.153501,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
276,277,0.349747,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
277,278,0.659868,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
278,279,0.389223,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0


In [406]:
main=['Match','Result','Playerno','_Afshin',
 '_Ali Asadinia',
 '_Ali Izadi',
 '_Sina Eslami',
 '_Payam Zh. Monfared',
 '_Mohsen Mirzaei',
 '_Milad',
 '_Alireza Najafi','_Amin','_Sina Nozarian']

In [470]:
dt3=dt3[main]
dt3

,Match,Result,Playerno,_Afshin,_Ali Asadinia,_Ali Izadi,_Sina Eslami,_Payam Zh. Monfared,_Mohsen Mirzaei,_Milad,_Alireza Najafi,_Amin,_Sina Nozarian
0,1,0.473934,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
1,2,0.389223,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0
2,3,0.659868,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0
3,4,0.349747,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0
4,5,0.153501,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,276,0.153501,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0
276,277,0.349747,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0
277,278,0.659868,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0
278,279,0.389223,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0


In [477]:
dt3.sort_values("Result").head(15)

,Match,Result,Playerno,_Afshin,_Ali Asadinia,_Ali Izadi,_Sina Eslami,_Payam Zh. Monfared,_Mohsen Mirzaei,_Milad,_Alireza Najafi,_Amin,_Sina Nozarian
114,115,0.142823,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
165,166,0.142823,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
265,266,0.146975,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0
14,15,0.146975,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0
205,206,0.149359,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0
74,75,0.149359,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0
169,170,0.152195,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0
110,111,0.152195,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0
198,199,0.152739,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0
81,82,0.152739,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0


In [413]:
dic3={}
for i in dt3.columns[3:]:
    dic3[i]=dt[dt[i]==1]['Result'].mean()-0.5
    
Rating=dict(sorted(dic3.items(), key=lambda item: item[1]))

Rating

{'_Amin': -0.07507085267986568,
 '_Ali Asadinia': -0.0715882435441017,
 '_Afshin': -0.060122562944889046,
 '_Alireza Najafi': -0.0573952562042645,
 '_Milad': -0.010627275918211276,
 '_Payam Zh. Monfared': 0.004996110924652686,
 '_Ali Izadi': 0.023498413605349433,
 '_Sina Nozarian': 0.07507085267986568,
 '_Sina Eslami': 0.08107966972248892,
 '_Mohsen Mirzaei': 0.09015914435897554}

In [452]:
sum(Rating.values())

5.551115123125783e-17

In [454]:
r1={'_Amin': -0.07507085267986568,
 '_Ali Asadinia': -0.0615882435441017,
 '_Afshin': -0.060122562944889046,
 '_Alireza Najafi': -0.0573952562042645,
 '_Payam Zh. Monfared': -0.010627275918211276,
 '_Milad': 0.024996110924652686,
 '_Ali Izadi': 0.033498413605349433,
 '_Sina Eslami': 0.04107966972248892,
 '_Sina Nozarian': 0.07507085267986568,
 '_Mohsen Mirzaei': 0.09015914435897554}
r1

{'_Amin': -0.07507085267986568,
 '_Ali Asadinia': -0.0615882435441017,
 '_Afshin': -0.060122562944889046,
 '_Alireza Najafi': -0.0573952562042645,
 '_Payam Zh. Monfared': -0.010627275918211276,
 '_Milad': 0.024996110924652686,
 '_Ali Izadi': 0.033498413605349435,
 '_Sina Eslami': 0.04107966972248892,
 '_Sina Nozarian': 0.07507085267986568,
 '_Mohsen Mirzaei': 0.09015914435897554}

In [455]:
r1.values()

dict_values([-0.07507085267986568, -0.0615882435441017, -0.060122562944889046, -0.0573952562042645, -0.010627275918211276, 0.024996110924652686, 0.033498413605349435, 0.04107966972248892, 0.07507085267986568, 0.09015914435897554])

In [456]:
final=pd.DataFrame(r1.values(),columns=['Score'])
final['Name']=r1.keys()
final=final[['Name','Score']].sort_values('Score',ascending=False)
final['Score']=round(final['Score']*100,2)
final=final.set_index('Name')
final

,Score
Name,
_Mohsen Mirzaei,9.02
_Sina Nozarian,7.51
_Sina Eslami,4.11
_Ali Izadi,3.35
_Milad,2.50
_Payam Zh. Monfared,-1.06
_Alireza Najafi,-5.74
_Afshin,-6.01
_Ali Asadinia,-6.16


In [467]:
dt3.sort_values('Result')

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
114,115,0.142823,1.0,-1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,0.0
165,166,0.142823,-1.0,-1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,0.0
265,266,0.146975,-1.0,1.0,0.0,1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
14,15,0.146975,1.0,1.0,0.0,1.0,-1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,0.0
205,206,0.149359,-1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,215,0.850641,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
154,155,0.853025,-1.0,-1.0,0.0,-1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0
125,126,0.853025,1.0,-1.0,0.0,-1.0,1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0
254,255,0.857177,-1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,...,0.0,-1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
